### Курсовой проект для курса "Python для Data Science"

**Материалы к проекту (файлы):**
train.csv
test.csv

**Задание:**
Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

**Целевая переменная:**
Price

**Метрика:**
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

**Сдача проекта:**
1. Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту")
ссылку на программу в github (программа должна содержаться в файле Jupyter Notebook 
с расширением ipynb). (Pull request не нужен, только ссылка ведущая на сам скрипт).
2. Приложить файл с названием по образцу SShirkin_predictions.csv
с предсказанными ценами для квартир из test.csv (файл должен содержать два поля: Id, Price).
В файле с предсказаниями должна быть 5001 строка (шапка + 5000 предсказаний).

**Сроки и условия сдачи:**
Дедлайн: сдать проект нужно в течение 72 часов после начала Урока 10 ("Вебинар. Консультация по итоговому проекту").
Для успешной сдачи должны быть все предсказания (для 5000 квартир) и R2 должен быть больше 0.6.
При сдаче до дедлайна результат проекта может попасть в топ лучших результатов.
Повторная сдача и проверка результатов возможны только при условии предыдущей неуспешной сдачи.
Успешный проект нельзя пересдать в целях повышения результата.
Проекты, сданные после дедлайна или сданные повторно, не попадают в топ лучших результатов, но можно узнать результат.
В качестве итогового результата берется первый успешный результат, последующие успешные результаты не учитываются.

**Примечание:**
Все файлы csv должны содержать названия полей (header - то есть "шапку"),
разделитель - запятая. В файлах не должны содержаться индексы из датафрейма.

**Рекомендации для файла с кодом (ipynb):**
1. Файл должен содержать заголовки и комментарии
2. Повторяющиеся операции лучше оформлять в виде функций
3. Не делать вывод большого количества строк таблиц (5-10 достаточно)
4. По возможности добавлять графики, описывающие данные (около 3-5)
5. Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
6. Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
7. Весь проект должен быть в одном скрипте (файл ipynb).
8. При использовании статистик (среднее, медиана и т.д.) в качестве признаков,
лучше считать их на трейне, и потом на валидационных и тестовых данных не считать 
статистики заново, а брать их с трейна. Если хватает знаний, можно использовать кросс-валидацию,
но для сдачи этого проекта достаточно разбить данные из train.csv на train и valid.
9. Проект должен полностью отрабатывать за разумное время (не больше 10 минут),
поэтому в финальный вариант лучше не включать GridSearch с перебором 
большого количества сочетаний параметров.
10. Допускается применение библиотек Python и моделей машинного обучения,
которые были в курсе Python для Data Science. Градиентный бустинг изучается
в последующих курсах, поэтому в этом проекте его применять не следует.
Самая сложная из допустимых моделей - RandomForestRegressor из sklearn.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from pylab import rcParams

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

**Пути к директориям и файлам**

In [3]:
TRAIN_DATASET_PATH = 'train.csv'
TEST_DATASET_PATH = 'test.csv'

**Заргузка данных**

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры

In [4]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
print(train_df.shape)
train_df.head()

(10000, 20)


,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [5]:
train_df.isna().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

**Замена символьных значений в категориальных переменных**

In [6]:
train_df.replace({'Ecology_2':{'A':0, 'B':1}, 'Ecology_3':{'A':0, 'B':1}, 'Shops_2':{'A':0, 'B':1}}, inplace=True)
train_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,1,1,33,7976,5,NaN,0,11,1,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,1,1,46,10309,1,240.0,1,16,1,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,1,1,34,7759,0,229.0,1,3,1,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,1,1,23,5735,3,1084.0,0,5,1,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,1,1,35,5776,1,2078.0,2,4,1,150226.531644


**Разбиение датафрейма на трейн и валид**

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split (train_df.set_index('Id').drop(['LifeSquare', 'Healthcare_1', 'Price'], axis='columns'), train_df['Price'], test_size=0.2, random_state=22)

**Обработка выбросов и заполнение пропусков**

In [8]:
train_df['Rooms'].value_counts()

2.0     3880
1.0     3705
3.0     2235
4.0      150
5.0       18
0.0        8
10.0       2
19.0       1
6.0        1
Name: Rooms, dtype: int64

In [9]:
train_df.replace({'Rooms':[train_df['Rooms'][(train_df['Rooms'] < 1) | (train_df['Rooms'] > 5)]]}, 2, inplace=True)
train_df['Rooms'].value_counts()

2.0    3892
1.0    3705
3.0    2235
4.0     150
5.0      18
Name: Rooms, dtype: int64

In [10]:
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [11]:
test_df.replace({'Ecology_2':{'A':0, 'B':1}, 'Ecology_3':{'A':0, 'B':1}, 'Shops_2':{'A':0, 'B':1}}, inplace=True)
test_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,1,1,11,2748,1,NaN,0,0,1
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,1,1,6,1437,3,NaN,0,2,1
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,1,1,30,7538,87,4702.0,5,5,1
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,1,1,23,4583,3,NaN,3,3,1
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,1,1,2,629,1,NaN,0,0,0


In [12]:
test_df['Rooms'].value_counts()

2.0     2030
1.0     1769
3.0     1099
4.0       90
5.0        7
0.0        2
6.0        2
17.0       1
Name: Rooms, dtype: int64

In [13]:
test_df.replace({'Rooms':[train_df['Rooms'][(train_df['Rooms'] < 1) | (train_df['Rooms'] > 6)]]}, 2, inplace=True)

**Обработка Helthcare_1**

In [14]:
train_cls_df = train_df[['DistrictId', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1']].dropna()
train_cls_df

,DistrictId,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1
1,41,0.000070,46,10309,1,240.0
2,53,0.049637,34,7759,0,229.0
3,58,0.437885,23,5735,3,1084.0
4,99,0.012339,35,5776,1,2078.0
5,59,0.309479,35,7715,4,990.0
...,...,...,...,...,...,...
9990,1,0.036270,6,1318,1,200.0
9991,3,0.265089,37,5288,0,1937.0
9995,32,0.135650,46,7960,6,350.0
9998,75,0.307467,30,5048,9,325.0


In [15]:
X_cls_train, X_cls_valid, y_cls_train, y_cls_valid = train_test_split (train_cls_df.drop(['Healthcare_1'], axis='columns'), train_cls_df['Healthcare_1'], test_size=0.2, random_state=22)

In [16]:
model_RFC = RandomForestClassifier(max_depth=9, n_estimators=200, random_state=55)
model_RFC.fit(X_cls_train, y_cls_train)
train_cls_first_predict = model_RFC.predict(X_cls_train)
valid_cls_first_predict = model_RFC.predict(X_cls_valid)
print(f'{r2(y_cls_train, train_cls_first_predict)}')
print(f'{r2(y_cls_valid, valid_cls_first_predict)}')

0.8292238558111379
0.7967401976480248


In [17]:
y_cls_pred = model_RFC.predict(train_df[['DistrictId', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3']])

In [18]:
train_df['Healthcare_1_pred']=y_cls_pred
train_df.drop('Healthcare_1', axis='columns',inplace=True)
train_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,Healthcare_1_pred
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,1,1,33,7976,5,0,11,1,184966.930730,350.0
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,1,1,46,10309,1,1,16,1,300009.450063,240.0
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,1,1,34,7759,0,1,3,1,220925.908524,229.0
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,1,1,23,5735,3,0,5,1,175616.227217,1084.0
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,1,1,35,5776,1,2,4,1,150226.531644,2078.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,1,1,46,7960,6,3,11,1,196684.316040,350.0
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,1,1,30,5562,0,0,5,0,189050.289571,645.0
9997,5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,1,1,2,629,1,0,0,0,159143.805370,0.0
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961,0.307467,1,0,30,5048,9,2,5,1,181595.339808,325.0


**Helthcare_1 для тестовых данных**

In [19]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5000 non-null   int64  
 1   DistrictId     5000 non-null   int64  
 2   Rooms          5000 non-null   float64
 3   Square         5000 non-null   float64
 4   LifeSquare     3959 non-null   float64
 5   KitchenSquare  5000 non-null   float64
 6   Floor          5000 non-null   int64  
 7   HouseFloor     5000 non-null   float64
 8   HouseYear      5000 non-null   int64  
 9   Ecology_1      5000 non-null   float64
 10  Ecology_2      5000 non-null   int64  
 11  Ecology_3      5000 non-null   int64  
 12  Social_1       5000 non-null   int64  
 13  Social_2       5000 non-null   int64  
 14  Social_3       5000 non-null   int64  
 15  Healthcare_1   2623 non-null   float64
 16  Helthcare_2    5000 non-null   int64  
 17  Shops_1        5000 non-null   int64  
 18  Shops_2 

In [20]:
y_cls_pred_test = model_RFC.predict(test_df[['DistrictId', 'Ecology_1', 'Social_1', 'Social_2', 'Social_3']])
test_df['Healthcare_1_pred']=y_cls_pred_test
test_df.drop('Healthcare_1', axis='columns',inplace=True)
test_df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Healthcare_1_pred
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,1,1,11,2748,1,0,0,1,80.0
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,1,1,6,1437,3,0,2,1,200.0
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,1,1,30,7538,87,5,5,1,4702.0
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,1,1,23,4583,3,3,3,1,520.0
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,1,1,2,629,1,0,0,0,0.0


### Введение нового признака ###

In [21]:
median_price_by_id = train_df.groupby('DistrictId')[['Price']].median().reset_index()
median_price_by_id = median_price_by_id.rename({'Price':'Price_median'}, axis='columns')
median_price_by_id

,DistrictId,Price_median
0,0,165963.054142
1,1,183663.443595
2,2,208539.501373
3,3,169094.013281
4,4,278639.482329
...,...,...
200,202,394150.861857
201,205,220501.566180
202,207,426186.409334
203,208,431137.654083


In [22]:
train_df = pd.merge(train_df, median_price_by_id, how='left', on='DistrictId')
train_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,Healthcare_1_pred,Price_median
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,...,1,33,7976,5,0,11,1,184966.930730,350.0,203602.408898
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,...,1,46,10309,1,1,16,1,300009.450063,240.0,210694.850106
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,...,1,34,7759,0,1,3,1,220925.908524,229.0,245978.794474
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,...,1,23,5735,3,0,5,1,175616.227217,1084.0,151557.904767
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,...,1,35,5776,1,2,4,1,150226.531644,2078.0,178829.166450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,...,1,46,7960,6,3,11,1,196684.316040,350.0,234647.811956
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,...,1,30,5562,0,0,5,0,189050.289571,645.0,188489.490556
9997,5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,...,1,2,629,1,0,0,0,159143.805370,0.0,146171.433190
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961,0.307467,...,0,30,5048,9,2,5,1,181595.339808,325.0,194932.010500


In [23]:
X_train, X_valid, y_train, y_valid = train_test_split (train_df.set_index('Id').drop(['LifeSquare', 'Price', 'DistrictId'], axis='columns'), train_df['Price'], test_size=0.2, random_state=22)

In [24]:
X_train.isna().sum()

Rooms                0
Square               0
KitchenSquare        0
Floor                0
HouseFloor           0
HouseYear            0
Ecology_1            0
Ecology_2            0
Ecology_3            0
Social_1             0
Social_2             0
Social_3             0
Helthcare_2          0
Shops_1              0
Shops_2              0
Healthcare_1_pred    0
Price_median         0
dtype: int64

In [25]:
model_RFR = RandomForestRegressor(max_depth=10, n_estimators=200, random_state=31)
model_RFR.fit(X_train, y_train)
train_first_predict = model_RFR.predict(X_train)
valid_first_predict = model_RFR.predict(X_valid)
print(f'{r2(y_train, train_first_predict)}')
print(f'{r2(y_valid, valid_first_predict)}')

0.8810084826293811
0.7243613976577628


**Вводим новый признак в тестовые данные**

In [26]:
test_df = pd.merge(test_df, median_price_by_id, how='left', on='DistrictId')
test_df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Healthcare_1_pred,Price_median
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,1,1,11,2748,1,0,0,1,80.0,151557.904767
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,1,1,6,1437,3,0,2,1,200.0,195610.960042
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,1,1,30,7538,87,5,5,1,4702.0,526438.458919
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,1,1,23,4583,3,3,3,1,520.0,196429.659238
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,1,1,2,629,1,0,0,0,0.0,146171.433190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8180,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,1,1,36,5992,0,1,1,1,645.0,202852.914457
4996,4695,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,1,1,1,264,0,0,1,1,540.0,183663.443595
4997,5783,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,1,1,74,19083,2,5,15,1,1322.0,201169.579060
4998,4780,62,2.0,81.305222,NaN,0.0,4,0.0,1977,0.072158,1,1,2,629,1,0,0,0,0.0,162067.275050


In [27]:
test_df['DistrictId'][test_df['Price_median'].isna()].value_counts()

206    2
204    1
203    1
198    1
212    1
211    1
210    1
Name: DistrictId, dtype: int64

In [28]:
median_price_by_id['Price_median'].median()

233005.92599930058

In [29]:
test_df['Price_median'] = test_df['Price_median'].fillna(median_price_by_id['Price_median'].median())

In [30]:
test_df['Price_median'].isna().sum()

0

In [31]:
test_first_predict = model_RFR.predict(test_df.set_index('Id').drop(['LifeSquare', 'DistrictId'], axis='columns'))
test_df['Price'] = test_first_predict

In [32]:
test_df[['Id', 'Price']].to_csv('IGromykhin_predict_18042021_test6.csv', sep=',', index=False)